<a href="https://colab.research.google.com/github/PTC-Education/PTC-Education.github.io/blob/master/MicroBit_Onshape_Digital_Twin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Onshape Digital Twin with Microbit

Public Onshape document of the Microbit [linked here](https://cad.onshape.com/documents/32703d124184c7b55761b09f/w/d7eb9109a3312be255719dc1/e/857afb03b2be711fe203dcbb). Microbit code for sending accelerometer values over the serial port [linked here](https://makecode.microbit.org/_3LdM96bVWWgX). First download the code onto your Microbit, then unplug the microbit and close the webpage. When connecting over serial, you may need to eject the device you see as a removable drive in order to use 

## Start Local Runtime

Follow the [instructions here](https://research.google.com/colaboratory/local-runtimes.html) for connecting Colab to a local runtime, or download this notebook and run from Jupyter.

## Import serial library and define functions

In [ ]:
import serial
import sys
import glob
import time

def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
        for port in ports:
            if 'usb' in port:
                guess = port
        return guess
    else:
        raise EnvironmentError('Unsupported platform')
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return ports

def serial_write(string):
    ser.write(string + b'\r\n')
    time.sleep(0.1)
    while ser.in_waiting:  
        print(ser.read(ser.in_waiting).decode())
        

Should return "True" if the device is connected

In [ ]:
port = serial_ports()
ser = serial.Serial(
    port=port,
    baudrate=9600
)

ser.isOpen()

### Define function for getting the accelerometer values into the correct range for the Onshape digital twin

In [ ]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

## Import Onshape Client and configure with API Keys

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

Change filepath below to file with your keys. Should be of the form:
```
access = 'access-key-here'
secret = 'secret-key-here'
```

In [ ]:
%run '~/Documents/colabkeys.py'
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
print('client configured')

## Define Onshape Functions
Make sure you update the DID, WID, and EID to the **assembly** of your Microbit digital twin.

### Transform Onshape model function

In [ ]:
def rotation(Rx,Ry,Rz):
  ## Get assembly occurances
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid'

  # https://cad.onshape.com/documents/32703d124184c7b55761b09f/w/d7eb9109a3312be255719dc1/e/857afb03b2be711fe203dcbb
  did = '32703d124184c7b55761b09f'
  wid = 'd7eb9109a3312be255719dc1'
  eid = '857afb03b2be711fe203dcbb'

  method = 'GET'

  params = {'includeNonSolids':True}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  # print(response.data)

  occurrences = json.loads(response.data)['rootAssembly']['occurrences']

  for x in occurrences:
      if x['path'][0] == 'MDO8kJV4gZ65wezUr':
        diceOccurrence = x
        print(diceOccurrence)

  # Translate object by x1,y1,z1 in the x,y,z directions
  mat = fourByFourToSixteen(numpy.matmul(clockwiseSpinY(Ry),clockwiseSpinX(Rx)))

  diceOccurrence['transform'] = mat

  ## Send assembly occurence transforms
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/occurrencetransforms'

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  method = 'POST'

  params = {}
  payload = {'isRelative':False,
            'occurrences':[diceOccurrence],
            'transform':mat}

  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)


In [ ]:
rotation(0.1,0.1,0)

### Matrix Library

In [ ]:
import math
import numpy

def IdentitySixteen():
  m = [
      1, 0, 0, 0,
      0, 1, 0, 0,
      0, 0, 1, 0,
      0, 0, 0, 1
      ]
  return m

def DiceMove(value,x1,y1):
  matrix = DiceTransform(value)
  matrix[3] = x1
  matrix[7] = y1
  return matrix

def DiceFollow(diceMatrix,tcpMatrix):
  diceMatrix[3] = tcpMatrix[3]
  diceMatrix[7] = tcpMatrix[7]
  diceMatrix[11] = tcpMatrix[11]
  return diceMatrix
  
def sixteenToFourByFour(matrix):
  fourbyfour = [[matrix[0],matrix[1],matrix[2],matrix[3]],
                [matrix[4],matrix[5],matrix[6],matrix[7]],
                [matrix[8],matrix[9],matrix[10],matrix[11]],
                [matrix[12],matrix[13],matrix[14],matrix[15]]]
  return fourbyfour

def fourByFourToSixteen(matrix):
  sixteen = [matrix[0][0],matrix[0][1],matrix[0][2],matrix[0][3],
             matrix[1][0],matrix[1][1],matrix[1][2],matrix[1][3],
             matrix[2][0],matrix[2][1],matrix[2][2],matrix[2][3],
             matrix[3][0],matrix[3][1],matrix[3][2],matrix[3][3]]
  return sixteen

def clockwiseSpinX(theta):
  m = [[1, 0, 0, 0],
       [0, math.cos(theta), math.sin(theta), 0],
       [0, -math.sin(theta), math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m

def clockwiseSpinY(theta):
  m = [[math.cos(theta), 0, math.sin(theta), 0],
       [0, 1, 0, 0],
       [-math.sin(theta), 0, math.cos(theta), 0],
       [0, 0, 0, 1]
       ]
  return m

def clockwiseSpinZ(theta):
  m = [[math.cos(theta), math.sin(theta), 0, 0],
       [-math.sin(theta), math.cos(theta), 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]]
  return m

# Main loop

In [ ]:
import time
while True:
  ser.flushInput()
  accelerometer = ser.readline().decode()
  accelerometer = accelerometer.strip()
  try:
    rx,ry,rz = accelerometer.split(',')
    print(rx,ry,rz)
    rz = int(rz)
    if rz < 1:
      OnshapeRY = translate(int(rx),-1024,1024,-math.pi/2,math.pi/2)
      OnshapeRX = translate(int(ry),-1024,1024,math.pi/2,-math.pi/2)
      print(OnshapeRX,OnshapeRY)
    elif rz > 1:
      OnshapeRY = translate(int(rx),-1024,1024,math.pi/2,3*math.pi/2)
      OnshapeRX = translate(int(ry),-1024,1024,-math.pi/2,math.pi/2)
      print(OnshapeRX,OnshapeRY)
    rotation(OnshapeRX,OnshapeRY,OnshapeRZ)
  except:
    print('mid serial send')